### Exploratory Data Analysis

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [3]:
df = pd.read_csv("../data/1_work_data/processed_data/control.csv")

In [4]:
missing_values = df.isnull().sum().sort_values(ascending=False)
print("\nMissing Values Count:\n", missing_values)


Missing Values Count:
 ltda_yoy     44
ni_2y        24
ev_eb_yoy    21
pe_yoy       21
freturn      15
             ..
n_sales       0
val_purch     0
n_purch       0
tdq           0
sector        0
Length: 103, dtype: int64


### Visualization

In [5]:
aapl_df = df[df.tic == 'AAPL']

In [9]:
fig = px.bar(aapl_df, x='rdq', y='der')
fig.show()

In [10]:
fig = px.bar(aapl_df, x='rdq', y='cr')
fig.show()

### Histograms

In [4]:
fig_saleq = px.histogram(df, x='saleq', nbins=50, marginal='box', title='Distribution of Sales (saleq)')
fig_saleq.show()